# Segmentation

In [5]:
import os
from modalities.test_mask_segment import  segment_brain_images, mask_brain_images
from modalities.report_segmentation import traverse_and_generate_report

def run_segmentation_and_masking(input_folder: str, tpm_path: str):
    # searching for .nii files within the subdirectories of input_folder
    subject_ids = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.nii'):
                subject_ids.append(os.path.join(root, file))
    
    print(subject_ids)  # check .nii files

    for input_file in subject_ids:
        subject_id = os.path.basename(input_file).split('.')[0]
        subject_folder = os.path.dirname(input_file)
        segmentation_folder = os.path.join(subject_folder, 'segmentation')
        os.makedirs(segmentation_folder, exist_ok=True)
        
        for iteration in range(1, 4):
            segmentation_output = segment_brain_images(input_file, tpm_path, iteration, subject_id, subject_folder)
            if not segmentation_output:
                print(f"Segmentation failed for {input_file} in iteration {iteration}")
                break
            mask_output = mask_brain_images(os.path.join(segmentation_folder, f'{iteration}_segment'), subject_folder, subject_id, iteration)
            input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'masked_{subject_id}.nii')
            
            # renaming the masked output to the original file name for the next iter
            next_input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'{subject_id}.nii') 
            os.rename(input_file, next_input_file)
            input_file = next_input_file
    
    # Generate the report after all subjects have been processed
    traverse_and_generate_report(input_folder, input_folder)

# Example usage:

# '/home/sharapova/raw_analysis/new_minori/T2'
input_folder = 'data/analysis_conditions/'
tpm_path = 'data/TPM'
run_segmentation_and_masking(input_folder, tpm_path)

['/home/sharapova/raw_analysis/new_minori/T2/Group/20240526_HF6_G2/T2_20240526_HF6_G2.nii', '/home/sharapova/raw_analysis/new_minori/T2/Group/20240526_HF6_G1/T2_20240526_HF6_G1.nii', '/home/sharapova/raw_analysis/new_minori/T2/Group/20240611_HF6_g4/T2_20240611_HF6_g4.nii', '/home/sharapova/raw_analysis/new_minori/T2/Group/20240612_HF6_g3/T2_20240612_HF6_g3.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240523_HF6_S2/T2_20240523_HF6_S2.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240522_HF6_S1/T2_20240522_HF6_S1.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240612_HF6_S4/T2_20240612_HF6_S4.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240524_HF6_S3/T2_20240524_HF6_S3.nii']
240719-09:13:21,342 nipype.workflow DEBUG:
	 (vbm_segment_1_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_1, vbm_segment_1_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_1): No edge data
240719-09:13:21,344 nipype.workflow DEBUG:
	 (vbm_segment_1_T2_20240526_HF6_G2.

stty: 'standard input': Inappropriate ioctl for device


240719-09:14:12,701 nipype.workflow INFO:
	 [Node] Finished "segment_T2_20240526_HF6_G1_1", elapsed time 23.714517999999998s.
240719-09:14:12,701 nipype.workflow DEBUG:
	 Saving results file: '/home/sharapova/raw_analysis/new_minori/T2/Group/20240526_HF6_G1/segmentation/1_segment/vbm_segment_1_T2_20240526_HF6_G1/segment_T2_20240526_HF6_G1_1/result_segment_T2_20240526_HF6_G1_1.pklz'
240719-09:14:12,702 nipype.workflow WARNING:
	 Storing result file without outputs
240719-09:14:12,703 nipype.workflow WARNING:
	 [Node] Error on "vbm_segment_1_T2_20240526_HF6_G1.segment_T2_20240526_HF6_G1_1" (/home/sharapova/raw_analysis/new_minori/T2/Group/20240526_HF6_G1/segmentation/1_segment/vbm_segment_1_T2_20240526_HF6_G1/segment_T2_20240526_HF6_G1_1)
240719-09:14:12,705 nipype.workflow ERROR:
	 Node segment_T2_20240526_HF6_G1_1 failed to run on host mri.
240719-09:14:12,705 nipype.workflow ERROR:
	 Saving crash info to /home/sharapova/vbm_pipline/crash-20240719-091412-sharapova-segment_T2_20240526_H

stty: 'standard input': Inappropriate ioctl for device


240719-09:14:39,114 nipype.workflow INFO:
	 [Node] Finished "segment_T2_20240611_HF6_g4_1", elapsed time 26.367998s.
240719-09:14:39,115 nipype.workflow DEBUG:
	 Saving results file: '/home/sharapova/raw_analysis/new_minori/T2/Group/20240611_HF6_g4/segmentation/1_segment/vbm_segment_1_T2_20240611_HF6_g4/segment_T2_20240611_HF6_g4_1/result_segment_T2_20240611_HF6_g4_1.pklz'
240719-09:14:39,115 nipype.workflow WARNING:
	 Storing result file without outputs
240719-09:14:39,118 nipype.workflow WARNING:
	 [Node] Error on "vbm_segment_1_T2_20240611_HF6_g4.segment_T2_20240611_HF6_g4_1" (/home/sharapova/raw_analysis/new_minori/T2/Group/20240611_HF6_g4/segmentation/1_segment/vbm_segment_1_T2_20240611_HF6_g4/segment_T2_20240611_HF6_g4_1)
240719-09:14:39,119 nipype.workflow ERROR:
	 Node segment_T2_20240611_HF6_g4_1 failed to run on host mri.
240719-09:14:39,120 nipype.workflow ERROR:
	 Saving crash info to /home/sharapova/vbm_pipline/crash-20240719-091439-sharapova-segment_T2_20240611_HF6_g4_1-d

...

240719-09:15:14,695 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240612_HF6_g3.segment_T2_20240612_HF6_g3_2, vbm_segment_2_T2_20240612_HF6_g3.data_sink_T2_20240612_HF6_g3_2): No edge data
240719-09:15:14,696 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240612_HF6_g3.segment_T2_20240612_HF6_g3_2, vbm_segment_2_T2_20240612_HF6_g3.data_sink_T2_20240612_HF6_g3_2): new edge data: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-09:15:14,697 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240612_HF6_g3.segment_T2_20240612_HF6_g3_2, vbm_segment_2_T2_20240612_HF6_g3.data_sink_T2_20240612_HF6_g3_2): Edge data exists: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-09:15:14,697 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240612_HF6_g3.segment_T2_20240612_HF6_g3_2, vbm_segment_2_T2_20240612_HF6_g3.data_sink_T2_20240612_HF6_g3_2): new edge data: {'connect': [('transformation_mat', 'transformation_mat'), ('native_class_images', 'native_class_images')]}
240719-09:1

stty: 'standard input': Inappropriate ioctl for device


240719-09:15:53,868 nipype.workflow INFO:
	 [Node] Finished "segment_T2_20240523_HF6_S2_1", elapsed time 24.216408s.
240719-09:15:53,869 nipype.workflow DEBUG:
	 Saving results file: '/home/sharapova/raw_analysis/new_minori/T2/Single/20240523_HF6_S2/segmentation/1_segment/vbm_segment_1_T2_20240523_HF6_S2/segment_T2_20240523_HF6_S2_1/result_segment_T2_20240523_HF6_S2_1.pklz'
240719-09:15:53,869 nipype.workflow WARNING:
	 Storing result file without outputs
240719-09:15:53,872 nipype.workflow WARNING:
	 [Node] Error on "vbm_segment_1_T2_20240523_HF6_S2.segment_T2_20240523_HF6_S2_1" (/home/sharapova/raw_analysis/new_minori/T2/Single/20240523_HF6_S2/segmentation/1_segment/vbm_segment_1_T2_20240523_HF6_S2/segment_T2_20240523_HF6_S2_1)
240719-09:15:53,873 nipype.workflow ERROR:
	 Node segment_T2_20240523_HF6_S2_1 failed to run on host mri.
240719-09:15:53,874 nipype.workflow ERROR:
	 Saving crash info to /home/sharapova/vbm_pipline/crash-20240719-091553-sharapova-segment_T2_20240523_HF6_S2_1

stty: 'standard input': Inappropriate ioctl for device


TraitError: Each element of the 'channel_files' trait of a NewSegmentInputSpec instance must be a pathlike object or string representing an existing file, but a value of '/home/sharapova/raw_analysis/new_minori/T2/Single/20240612_HF6_S4/T2_20240612_HF6_S4.nii' <class 'str'> was specified.

# Dartel

In [18]:
%load_ext autoreload
%autoreload 2
import os
from modalities.dartel import create_dartel_structure, perform_dartel, copy_segment_files, find_subject_id

def main(base_path: str):
    condition_folders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
    dartel_dir = create_dartel_structure(base_path)
    copy_segment_files(base_path, condition_folders, dartel_dir)
    perform_dartel(dartel_dir)

main('/home/sharapova/vbm_pipline/data/analysis_conditions')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/native_class_images/c1T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/native_class_images/c2T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c2/c2T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/dartel_input_images/rc1T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/rc1/rc1T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/dartel_input_images/rc2T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/

stty: 'standard input': Inappropriate ioctl for device


# Normalization

In [19]:
%load_ext autoreload
%autoreload 2
import os
from modalities.normalization import normalize_brain_images

root_derivatives_dir_path = os.path.abspath('/home/sharapova/vbm_pipline/data/analysis_conditions')
output_files = normalize_brain_images(root_derivatives_dir_path)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c1_files: ['/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_174.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_176.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_177.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_178.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_201.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_223.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_224.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_225.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_226.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_227.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_228.nii']
flow_field_files: ['/home/sharap

stty: 'standard input': Inappropriate ioctl for device


# Smoothing

In [20]:
%load_ext autoreload
%autoreload 2
import os
from modalities.smoothing import smooth_brain_images

root_derivatives_dir_path = os.path.abspath('/home/sharapova/vbm_pipline/data/analysis_conditions')

# Define the full width at half maximum (FWHM) for smoothing
fwhm = [3, 3, 3] 


output_files = smooth_brain_images(root_derivatives_dir_path, fwhm)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
normalize_files: ['/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_174.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_176.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_177.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_178.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_201.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_223.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_224.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_225.nii', '/home

240716-12:56:59,914 nipype.workflow DEBUG:
	 (vbm_smoothing.smoothing, vbm_smoothing.data_sink): new edge data: {'connect': [('smoothed_files', 'smoothed_files')]}
240716-12:56:59,914 nipype.workflow DEBUG:
	 Creating flat graph for workflow: vbm_smoothing
240716-12:56:59,918 nipype.workflow DEBUG:
	 expanding workflow: vbm_smoothing
240716-12:56:59,919 nipype.workflow DEBUG:
	 processing node: vbm_smoothing.smoothing
240716-12:56:59,919 nipype.workflow DEBUG:
	 processing node: vbm_smoothing.data_sink
240716-12:56:59,920 nipype.workflow DEBUG:
	 finished expanding workflow: vbm_smoothing
240716-12:56:59,920 nipype.workflow INFO:
	 Workflow vbm_smoothing settings: ['check', 'execution', 'logging', 'monitoring']
240716-12:56:59,923 nipype.workflow DEBUG:
	 PE: expanding iterables
240716-12:56:59,924 nipype.workflow DEBUG:
	 Detected iterable nodes []
240716-12:56:59,924 nipype.workflow DEBUG:
	 PE: expanding iterables ... done
240716-12:56:59,926 nipype.workflow DEBUG:
	 Performing dept

stty: 'standard input': Inappropriate ioctl for device


# Statistical modeling and estimation

In [3]:
%load_ext autoreload
%autoreload 2
import os
from modalities.statistical_modeling_t_test_model import create_two_sample_t_test_model

contrasts_name = 'example-contrast'
output_files = create_two_sample_t_test_model('/home/sharapova/vbm_pipline/data/analysis_conditions', contrasts_name)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Detected conditions: ['control_analysis', 'Treated_by_psilocin_analysis']
240716-14:46:30,220 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=4, mem_gb=28.10, cwd=/home/sharapova/vbm_pipline)


240716-14:46:30,221 nipype.workflow DEBUG:
	 Creating flat graph for workflow: vbm_stats_modeling
240716-14:46:30,226 nipype.workflow DEBUG:
	 expanding workflow: vbm_stats_modeling
240716-14:46:30,228 nipype.workflow DEBUG:
	 processing node: vbm_stats_modeling.vbm_stats_modeling
240716-14:46:30,228 nipype.workflow DEBUG:
	 processing node: vbm_stats_modeling.data_sink
240716-14:46:30,229 nipype.workflow DEBUG:
	 finished expanding workflow: vbm_stats_modeling
240716-14:46:30,229 nipype.workflow INFO:
	 Workflow vbm_stats_modeling settings: ['check', 'execution', 'logging', 'monitoring']
240716-14:46:30,232 nipype.workflow DEBUG:
	 PE: expanding iterables
240716-14:46:30,232 nipype.workflow DEBUG:
	 Detected iterable nodes []
240716-14:46:30,232 nipype.workflow DEBUG:
	 PE: expanding iterables ... done
240716-14:46:30,238 nipype.workflow DEBUG:
	 Performing depth first search
240716-14:46:30,241 nipype.workflow INFO:
	 Running in parallel.
240716-14:46:30,243 nipype.workflow DEBUG:
	 

# Contrast

In [4]:
%load_ext autoreload
%autoreload 2
import os
from modalities.statistical_analysis import perform_statistical_analysis

output_files = perform_statistical_analysis('/home/sharapova/vbm_pipline/data/analysis_conditions', 'control-treatment')
print("Output files:", output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Contrasts name provided: control-treatment
Parsed contrast1_name: control
Parsed contrast2_name: treatment
Constructed contrast1: ('control>treatment', 'T', ['Group_{1}', 'Group_{2}'], [1, -1])
Constructed contrast2: ('control<treatment', 'T', ['Group_{1}', 'Group_{2}'], [-1, 1])
240716-14:46:43,129 nipype.workflow DEBUG:
	 (vbm_stats_analysis.estimate_contrast, vbm_stats_analysis.data_sink): No edge data
240716-14:46:43,130 nipype.workflow DEBUG:
	 (vbm_stats_analysis.estimate_contrast, vbm_stats_analysis.data_sink): new edge data: {'connect': [('spm_mat_file', 'control-treatment.@spm_mat'), ('spmT_images', 'control-treatment.@T'), ('con_images', 'control-treatment.@con')]}
240716-14:46:43,131 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=4, mem_gb=28.10, cwd=/home/sharapova/vbm_pipline)
240716-14:46:43,132 nipype.workflow DEBUG:
	 Creating flat graph for workflow: vbm_stats_analysis
24071

240716-14:46:43,215 nipype.workflow DEBUG:
	 Allocating vbm_stats_analysis.estimate_contrast ID=0 (0.20GB, 1 threads). Free: 27.90GB, 3 threads.
240716-14:46:43,222 nipype.workflow DEBUG:
	 [Node] Hashes: [('beta_images', [('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/beta_0001.nii', 'ef31ccab1647af75d7d92934e175751e'), ('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/beta_0002.nii', '2d4a628aeb105f874ad79580db81df43')]), ('contrasts', [('control>treatment', 'T', ['Group_{1}', 'Group_{2}'], ['1.0000000000', '-1.0000000000']), ('control<treatment', 'T', ['Group_{1}', 'Group_{2}'], ['-1.0000000000', '1.0000000000'])]), ('group_contrast', True), ('mfile', True), ('residual_image', ('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/ResMS.nii', 'ac7fcc404090ddb2b120910cfcf6020b')), ('spm_mat_file', ('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/SPM.mat', '167db61c50a57dd7b315030b3b

stty: 'standard input': Inappropriate ioctl for device


240716-14:46:57,148 nipype.workflow INFO:
	 [Job 0] Completed (vbm_stats_analysis.estimate_contrast).
240716-14:46:57,151 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
240716-14:46:57,153 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.10/28.10, Free processors: 4/4.
240716-14:46:57,204 nipype.workflow DEBUG:
	 Allocating vbm_stats_analysis.data_sink ID=1 (0.20GB, 1 threads). Free: 27.90GB, 3 threads.
240716-14:46:57,210 nipype.workflow DEBUG:
	 [Node] Setting 3 connected inputs of node "data_sink" from 1 previous nodes.
240716-14:46:57,212 nipype.workflow DEBUG:
	 Resolving paths in outputs loaded from results file.
240716-14:46:57,212 nipype.workflow DEBUG:
	 output: spm_mat_file
240716-14:46:57,213 nipype.workflow DEBUG:
	 [Node] data_sink - setting input control-treatment.@spm_mat = /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/work/vbm_stats_analysis/estimate_contrast/SPM.mat
240716-14:46:57,213 nipype.wo

sh: 0: getcwd() failed: No such file or directory
stty: 'standard input': Inappropriate ioctl for device
